<a href="https://colab.research.google.com/github/amgito1648/clase-inteligencia-artificial/blob/main/IAIAIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial Segundo Corte de Inteligencia Artificial

Ralph Castellanos Couott - U00175144

In [ ]:
from sklearn.datasets import make_regression
import random
import numpy as np
import pandas as pd

# Configurar semilla para reproducibilidad
np.random.seed(42)
random.seed(42)

In [ ]:
# 2. Generación de datos sintéticos (valores positivos)
# =====================
X, y = make_regression(
    n_samples=300,
    n_features=10,
    noise=3.0,
    effective_rank=5,
    tail_strength=0.8,
    random_state=42
)

In [ ]:
# Desplazar todos los valores para asegurar que sean positivos
X = X - X.min(axis=0) + 1  # ahora todos los valores de X >= 1
y = y - y.min() + 1        # ahora todos los valores de y >= 1

In [ ]:
# Nombres descriptivos en español
nombres_columnas = [
    'edad', 'ingresos', 'años_educacion', 'tasa_empleo', 'densidad_poblacional',
    'temperatura_promedio', 'indice_salud', 'acceso_servicios',
    'nivel_urbanizacion', 'penetracion_tecnologica'
]

In [ ]:
# Crear DataFrame
df = pd.DataFrame(X, columns=nombres_columnas)
df['indice_desarrollo'] = y  # Variable objetivo
df.head()

In [ ]:
df.describe().T

In [ ]:
# Insertar >50% de valores nulos en una columna (e.g., 'indice_salud')
df.loc[:160, 'indice_salud'] = np.nan  # 161 valores nulos (~53%)

In [ ]:
# Insertar <5% de valores nulos en 2 columnas aleatorias
for col in ['edad', 'temperatura_promedio']:
    n_nulos = int(len(df) * 0.04)  # 4%
    indices = random.sample(range(len(df)), n_nulos)
    df.loc[indices, col] = np.nan

In [ ]:
# Insertar outliers en 2 columnas ('ingresos' y 'años_educacion')
outliers_ingresos = [5000, -4000, 6000, 7000, -3500]
df.loc[[5, 15, 25, 35, 45], 'ingresos'] = outliers_ingresos

In [ ]:
outliers_educacion = [50, 60, 70, -10]
df.loc[[60, 70, 80, 90], 'años_educacion'] = outliers_educacion

In [ ]:
# Insertar caracteres extraños en una columna y convertir a tipo object
df['acceso_servicios'] = df['acceso_servicios'].astype(str)
df.loc[[100, 120, 140], 'acceso_servicios'] = ['#¡VALOR!', '???', '%%error%%']

In [ ]:
# Verificar tipos de datos (opcional)
print(df.dtypes)

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
#Mirar cuántos valores atípicos hay en el DataFrame
df.isnull().sum()

In [ ]:
# Convertir a numérico los valores presentes en la columna acceso_servicios, ya que a diferencia de los demás, es el único de tipo object
df['acceso_servicios'] = pd.to_numeric(df['acceso_servicios'], errors='coerce')

# Eliminar filas con NaN en esa columna
df = df.dropna(subset=['acceso_servicios'])

In [ ]:
#Mirar que efectivamente ocurrió el cambio a tipo float64
print(df['acceso_servicios'].dtype)

In [ ]:
# 1. Eliminar la columna 'indice_salud' al tener más del 50% de valores nulos
porcentaje_nulos = df['indice_salud'].isna().mean()
if porcentaje_nulos > 0.5:
    df = df.drop(columns=['indice_salud'])

# 2. Imputar valores nulos con la mediana en las columnas que contienen valores atípicos (edad, temperatura_promedio)
for columna in ['edad', 'temperatura_promedio']:
    mediana = df[columna].median()
    df[columna] = df[columna].fillna(mediana)

In [ ]:
#Revisar que no haya quedado ningún valor atípico en el DataFrame
df.isnull().sum()

In [ ]:
def winsorizar_columna(df, columna, limite_inferior=0.01, limite_superior=0.99):
    p_inf = df[columna].quantile(limite_inferior)
    p_sup = df[columna].quantile(limite_superior)
    df[columna] = np.clip(df[columna], p_inf, p_sup)

# Aplicar winsorización a las columnas con outliers
winsorizar_columna(df, 'ingresos')
winsorizar_columna(df, 'años_educacion')

In [ ]:
df.dtypes

In [ ]:
def winsorizar_columna(df, columna, limite_inferior=0.01, limite_superior=0.99):
    p_inf = df[columna].quantile(limite_inferior)
    p_sup = df[columna].quantile(limite_superior)
    df[columna] = np.clip(df[columna], p_inf, p_sup)

# Aplicar a todas las columnas numéricas
columnas_numericas = df.select_dtypes(include=['float64']).columns

for col in columnas_numericas:
    winsorizar_columna(df, col, 0.01, 0.99)

In [ ]:
#Generar la caja de bigotes para cada columna
import matplotlib.pyplot as plt
import seaborn as sns

# Filtrar solo las columnas numéricas (float)
columnas_numericas = df.select_dtypes(include=['float64']).columns

# Crear un boxplot para cada columna numérica
for col in columnas_numericas:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col], color='skyblue')
    plt.title(f'Boxplot de {col}')
    plt.xlabel(col)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
#Generar un mapa de calor para visualizar la matriz de correlación

# Seleccionar variables numéricas (float)
variables_numericas = df.select_dtypes(include=["float"])

# Calcular la matriz de correlación
correlaciones = variables_numericas.corr()
# Crear el heatmap
plt.figure(figsize=(10, 8))  # Ajusta el tamaño según tu gusto
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Matriz de correlación entre variables numéricas')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

El mapa de calor nos indica que las columnas no están fuertemente colineadas entre ellos, ya que ninguna llega a ser igual o mayor del 80%

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
# Separar variables predictoras y objetivo
X = df.drop(columns='indice_desarrollo')
y = df['indice_desarrollo']

# División entrenamiento/prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
modelos = {
    "Regresión Lineal": (LinearRegression(), {}),
    "KNN": (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7]}),
    "SVR": (SVR(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
    "XGBoost": (XGBRegressor(verbosity=0), {'n_estimators': [100, 200], 'max_depth': [3, 5]}),
    "Random Forest": (RandomForestRegressor(), {'n_estimators': [100, 200], 'max_depth': [5, 10]})
}

In [ ]:
def evaluar_modelo(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # Cálculo manual
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return r2, rmse, mae, mape

resultados = {}

for nombre, (modelo, params) in modelos.items():
    grid = GridSearchCV(modelo, params, cv=5, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    r2, rmse, mae, mape = evaluar_modelo(y_test, y_pred)
    resultados[nombre] = {'R2': r2, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

    # Visualización: pred vs real
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_test, y=y_pred)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Real")
    plt.ylabel("Predicción")
    plt.title(f"{nombre} - Real vs Predicción")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Visualización: residuos
    residuales = y_test - y_pred
    plt.figure(figsize=(6, 4))
    sns.histplot(residuales, kde=True, bins=20)
    plt.title(f"{nombre} - Distribución de residuos")
    plt.xlabel("Error")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd

df_resultados = pd.DataFrame(resultados).T
print("Comparación de modelos:")
print(df_resultados.sort_values(by='R2', ascending=False))

Teniendo en cuenta los resultados y datos que muestra la comparación entre los 5 modelos (Regresión Lineal, KNN Regressor, SVR, XGBoost Regressor y Random Forest Regressor) el mejor fue el de SVR debido a lo siguiente:


*   Su R2 fue el más alto entre los modelos, de 85%, lo que indica que tuvo una mejor captura entre las variables.
*   Poseía el RMSE, MAE y MAPE más bajo entre todos



